In [ ]:
from imageai.Detection import ObjectDetection
import os
import cv2

In [ ]:
class Cartoonizer:
    """Cartoonizer effect
        A class that applies a cartoon effect to an image.
        The class uses a bilateral filter and adaptive thresholding to create
        a cartoon effect.
    """
    def __init__(self):
        pass

    def render(self, img_rgb):
        img_rgb = cv2.imread(img_rgb)
        numDownSamples = 2      # number of downscaling steps
        numBilateralFilters = 5  # number of bilateral filtering steps

        '''
        STEP 1 : 
        '''
        # Downsample image using Gaussian pyramid
        img_color = img_rgb
        for _ in range(numDownSamples):
            img_color = cv2.pyrDown(img_color)
        
        # Repeatedly apply small bilateral filter instead of applying one large filter
        for _ in range(numBilateralFilters):
            img_color = cv2.bilateralFilter(img_color, d=7,sigmaColor=9,sigmaSpace=9)
        
        # Upsample image to original size
        for _ in range(numDownSamples):
            img_color = cv2.pyrUp(img_color)
        
        '''
        STEP 2 AND 3
        '''
        # Convert to grayscale and apply median blur
        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        img_blur = cv2.medianBlur(img_gray, 3)
        
        '''
        STEP 4
        '''
        # Detect and enhance edges
        img_edge = cv2.adaptiveThreshold(img_blur, 255,
                                         cv2.ADAPTIVE_THRESH_MEAN_C,
                                         cv2.THRESH_BINARY, 
                                         blockSize = 9, 
                                         C = 7)
        
        '''
        STEP 5
        '''
        # Convert back to color so that it can be bit-ANDed with color image
        (x,y,z) = img_color.shape
        img_edge = cv2.resize(img_edge,(y,x)) 
        img_edge = cv2.cvtColor(img_edge, cv2.COLOR_GRAY2RGB)
        
        return cv2.bitwise_and(img_color, img_edge)

In [ ]:
execution_path = os.getcwd()

detector = ObjectDetection()
detector.setModelTypeAsRetinaNet()
detector.setModelPath(os.path.join(execution_path , "retinanet_best_coco.h5"))
detector.loadModel()

In [ ]:
# Specify input image name
input_image_path = os.path.join('Input', 'APJ-Kalam.JPG')

detections, extracted_images = detector.detectObjectsFromImage(input_image=os.path.join(execution_path , input_image_path), extract_detected_objects=True)

img = cv2.imread(input_image_path) 
tmp_canvas = Cartoonizer()

In [ ]:
i = 0
l =[]

for eachObject in detections:
    i = i + 1
    if(eachObject["name"] == 'person'):
        print(eachObject["name"] , " : " , eachObject["percentage_probability"])
        print ('Top left and bottom right coordinates of bounding box are: ', l)
        l = eachObject['box_points']
        
        object_img_path = eachObject["name"]+'-'+str(i)+'.jpg'
        crop_img = cv2.imread("-objects/"+object_img_path)
        cv2.imwrite('Output/crop_img'+str(i)+'.jpg', crop_img)
        
        result = tmp_canvas.render('Output/crop_img'+str(i)+'.jpg')
        cv2.imwrite('Output/cartoon_version'+str(i)+'.jpg', result)
        
        img[l[1]: l[1]+ result.shape[0], l[0]: l[0]+ result.shape[1]] = result
        cv2.imwrite('Output/final.png', img)